In [4]:
!pip install sklearn

In [61]:
!pip install -U 'scikit-learn<0.23'

Requirement already up-to-date: scikit-learn<0.23 in /home/lehoa/.local/lib/python3.8/site-packages (0.22.2.post1)


In [5]:
!pip  install sklearn-crfsuite


In [6]:
!pip install pandas

In [7]:
!pip install matplotlib

In [8]:
!pip install nltk

In [9]:
!pip install seaborn

In [62]:
import numpy as np
import pandas as pd 
import requests
from matplotlib import pyplot
import nltk, re, pprint
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from sklearn_crfsuite import CRF
from sklearn_crfsuite import metrics
from sklearn_crfsuite import scorers
from collections import Counter
nltk.download("treebank")

[nltk_data] Downloading package treebank to /home/lehoa/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [11]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/lehoa/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

 dataset: TreeBank Corpus. This tag set has 12 unique POS Tags

In [12]:
tagged_sentence = nltk.corpus.treebank.tagged_sents(tagset='universal')
print(len(tagged_sentence))

3914


In [13]:
tagged_words = []
for sen in tagged_sentence:
    for word in sen:
        tagged_words.append(word)
print(len(tagged_words))

100676


In [15]:
vocal = set(word for word, tag in tagged_words)
tag = set(tag for word, tag in tagged_words)

print("Len vocal: ", len(vocal))
print("Len tag: ", len(tag))

Len vocal:  12408
Len tag:  12


## Split data into train and test set : 80 - 20

In [16]:
train_set, test_set = train_test_split(tagged_sentence, test_size=20, random_state=40)
print("Number of Sentences in Train set: ", len(train_set))
print("NUmber of Sentences in Test set: ", len(test_set))

Number of Sentences in Train set:  3894
NUmber of Sentences in Test set:  20


In [36]:
train_set[0]

[('Mr.', 'NOUN'),
 ('Edelman', 'NOUN'),
 ('declined', 'VERB'),
 ('*-2', 'X'),
 ('to', 'PRT'),
 ('specify', 'VERB'),
 ('what', 'PRON'),
 ('*T*-1', 'X'),
 ('prompted', 'VERB'),
 ('the', 'DET'),
 ('recent', 'ADJ'),
 ('moves', 'NOUN'),
 (',', '.'),
 ('*-2', 'X'),
 ('saying', 'VERB'),
 ('0', 'X'),
 ('they', 'PRON'),
 ('are', 'VERB'),
 ('meant', 'VERB'),
 ('only', 'ADV'),
 ('*-4', 'X'),
 ('to', 'PRT'),
 ('benefit', 'VERB'),
 ('shareholders', 'NOUN'),
 ('when', 'ADV'),
 ('``', '.'),
 ('the', 'DET'),
 ('company', 'NOUN'),
 ('is', 'VERB'),
 ('on', 'ADP'),
 ('a', 'DET'),
 ('roll', 'NOUN'),
 ('*T*-3', 'X'),
 ('.', '.'),
 ("''", '.')]

In [31]:
def features(sentence, index):
    return{
        "is_first_capital: ": int(sentence[index][0].isupper()),  # is the first capital
        "is_first_word: ": int(index==0), #the first word in the sentence
        "is_last_word:": int(index == len(sentence) - 1), 
        "is_complete_capital: ": int(sentence[index].upper() == sentence[index]), 
         'prev_word':'' if index==0 else sentence[index-1],
        'next_word':'' if index==len(sentence)-1 else sentence[index+1],
        'is_numeric':int(sentence[index].isdigit()),
        'is_alphanumeric': int(bool((re.match('^(?=.*[0-9]$)(?=.*[a-zA-Z])',sentence[index])))),
        'prefix_1':sentence[index][0],
        'prefix_2': sentence[index][:2],
        'prefix_3':sentence[index][:3],
        'prefix_4':sentence[index][:4],
        'suffix_1':sentence[index][-1],
        'suffix_2':sentence[index][-2:],
        'suffix_3':sentence[index][-3:],
        'suffix_4':sentence[index][-4:],
        'word_has_hyphen': 1 if '-' in sentence[index] else 0
    }

In [32]:
def untag(sentence):
    return [word for word, tag in sentence]

def prepareData(tagged_sentences):
    X, y = [], []
    for sentences in tagged_sentences:
        X.append([features(untag(sentences), index) for index in range(len(sentences))])
        y.append([tag  for word, tag in sentences])
    return X, y

In [34]:
X_train, y_train = prepareData(train_set)
X_test, y_test = prepareData(test_set)

In [35]:
X_train[0]

[{'is_first_capital: ': 1,
  'is_first_word: ': 1,
  'is_last_word:': 0,
  'is_complete_capital: ': 0,
  'prev_word': '',
  'next_word': 'Edelman',
  'is_numeric': 0,
  'is_alphanumeric': 0,
  'prefix_1': 'M',
  'prefix_2': 'Mr',
  'prefix_3': 'Mr.',
  'prefix_4': 'Mr.',
  'suffix_1': '.',
  'suffix_2': 'r.',
  'suffix_3': 'Mr.',
  'suffix_4': 'Mr.',
  'word_has_hyphen': 0},
 {'is_first_capital: ': 1,
  'is_first_word: ': 0,
  'is_last_word:': 0,
  'is_complete_capital: ': 0,
  'prev_word': 'Mr.',
  'next_word': 'declined',
  'is_numeric': 0,
  'is_alphanumeric': 0,
  'prefix_1': 'E',
  'prefix_2': 'Ed',
  'prefix_3': 'Ede',
  'prefix_4': 'Edel',
  'suffix_1': 'n',
  'suffix_2': 'an',
  'suffix_3': 'man',
  'suffix_4': 'lman',
  'word_has_hyphen': 0},
 {'is_first_capital: ': 0,
  'is_first_word: ': 0,
  'is_last_word:': 0,
  'is_complete_capital: ': 0,
  'prev_word': 'Edelman',
  'next_word': '*-2',
  'is_numeric': 0,
  'is_alphanumeric': 0,
  'prefix_1': 'd',
  'prefix_2': 'de',
  'pr

In [37]:
y_train[0]

['NOUN',
 'NOUN',
 'VERB',
 'X',
 'PRT',
 'VERB',
 'PRON',
 'X',
 'VERB',
 'DET',
 'ADJ',
 'NOUN',
 '.',
 'X',
 'VERB',
 'X',
 'PRON',
 'VERB',
 'VERB',
 'ADV',
 'X',
 'PRT',
 'VERB',
 'NOUN',
 'ADV',
 '.',
 'DET',
 'NOUN',
 'VERB',
 'ADP',
 'DET',
 'NOUN',
 'X',
 '.',
 '.']

## Using a CRF model with the default Parameters

In [63]:
crf = CRF(
    algorithm='lbfgs',
    c1=0.01,
    c2=0.1,
    max_iterations=100,
    keep_tempfiles=None,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)


AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'